# 1. polglot-ko-1.3b를 squarelike/sharegpt_deepl_ko_translation로 파인 튜닝한 모델

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "aeolian83/Gugugo_for_DnD_v0.7"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [2]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [3]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [4]:
def gen(lan="en", x=""):
    if (lan == "ko"):
        prompt = f"### 한국어: {x}</끝>\n### 영어:"
    else:
        prompt = f"### 영어: {x}</끝>\n### 한국어:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=2048,
        temperature=0.001,
        no_repeat_ngram_size=10,
        early_stopping=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    return tokenizer.decode(gened[0]).replace(prompt+" ", "")

In [5]:
gen(lan="en", x="The Steward informed me that a mysterious package has somehow arrived on my ship. I should speak with her to learn more.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'집사가 내게 신비한 꾸러미가 내 배에 도착했다고 알려주었다. 그녀와 더 많은 것을 배우기 위해 그녀와 이야기를 해봐야겠다.</끝>'

In [6]:
gen(lan="en", x="While you are wearing armor, you gain a +1 bonus to AC.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'갑옷을 입고 있는 동안에는 AC에 +1의 보너스를 얻습니다.</끝>'

In [10]:
gen(lan="en", x="Chultans are the human natives of Chult. They have dark skin, black hair, and a rich culture. Driven out of the jungle by monsters and undead, they took refuge behind the stout walls of Port Nyanzaru, gave up their dynasties, and united to become a mercantile power in the southern ocean, eager to do business with anyone who visits their perilous land. A few Chultans migrated northward and established outposts in dis tant cities such as Athkatla, Baldur's Gate, Cali mport, Memnon, and Zazesspur.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'컬트는 컬트의 인간들입니다. 그들은 검은 피부, 검은 머리카락, 풍부한 문화를 가지고 있습니다. 괴물과 언데드에 의해 사막을 방황하던 그들은 포니족의 벽 뒤에 피난처를 만들어, 포니족의 왕족들을 쫓아내고, 남부 해안의 항구 니야자루의 거대한 벽 뒤에 피난처를 만들어, 그들은 자신들의 위험한 땅을 방문하는 사람들에게 사업을 하기를 원하는, 위험한 땅을 방문하는 사람들을 위한 상인으로서의 힘을 가진 자들로 가득 차 있습니다. 몇몇의 컬트들이 북쪽으로 이주하여 아스카틀라, 발더스 게이트, 칼림나, 엠노, 그리고 자지뿌러 같은 도시들에 정착했습니다.</끝>'

In [8]:
# gen(lan="en", x="")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'나는 어떤 식으로든 구경꾼들과 함께 여행을 하고 있는 것 같습니다. <b>Humaire</B>는 이곳의 관리인이라고 하더군요. 이상한 초대에 대한 답을 얻을 수 있을지도 모릅니다. 그녀는 현재 동쪽에 있습니다.</끝>'

In [7]:
gen(lan="en", x="<b>Humaire</b> told me to speak with her in the <b>Hall of Memories</b> on the <b>west side of the temple</b> beneath the arena.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<b>후마이리</b>가 <b>사원의 서쪽</b>에 있는 <b>기억의 전당</b>에서 그녀와 이야기하라고 했다.</끝>'